In [1]:
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
from ipywidgets import interact, Layout
import cvzone
import time
import threading
import multiprocessing as mp
from multiprocessing.shared_memory import SharedMemory
from sklearn.model_selection import train_test_split
import t2
import pickle

In [2]:
with open('model_0.pkl', 'rb') as file:  
    clf = pickle.load(file)

In [3]:
target_dict = {
    'Karta Polaka (FRONT)': 0,
    'Karta Polaka (BACK)': 1,
    'Driver License (FRONT)': 2,
    'Driver License (BACK)': 3,
              }

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX
width_img = 1280
height_img = 960

cap = cv2.VideoCapture(0)
display_handle=display(None, display_id=True)
# s = cap.read()[1].shape

tmp = mp.Value('i', 0)
contours_size = mp.Value('i', 0)
len_contours = mp.Value('i', 0)
# frame_bytes = s[0] * s[1] * s[2]

frame_memory = SharedMemory(name='FrameMemory', create=True, size=width_img*height_img*3)
contours_memory = SharedMemory(name='ContoursMemory', create=True, size=4096)

p1 = mp.Process(target=t2.my_process, args=(tmp, width_img, height_img, len_contours, contours_size,), daemon=True)
p1.start()

stopButton = widgets.ToggleButton(description='Stop', disabled=False)

def preprocess(fr):
    mask = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    mask = cv2.GaussianBlur(mask, (5,5), 2)
    mask = cv2.Canny(mask, 30, 255, apertureSize=5)
    return mask

def contours(fr_m, fr):
    biggest = np.array([])
    max_area = 0
    contours, hierarchy = cv2.findContours(fr_m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100_000:
            epsilon = 0.1 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, epsilon, True)
            cv2.drawContours(fr, cnt, -1, (0,255,0), 5)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest

def reorder(my_points):
    my_points = my_points.reshape((4,2))
    my_points_new = np.zeros((4,1,2), np.int32)
    add = my_points.sum(1)

    my_points_new[0] = my_points[np.argmin(add)]
    my_points_new[3] = my_points[np.argmax(add)]

    diff = np.diff(my_points, axis=1)

    my_points_new[1] = my_points[np.argmin(diff)]
    my_points_new[2] = my_points[np.argmax(diff)]

    return my_points_new

def get_warp(img, approx):
    biggest = reorder(approx)

    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [width_img, 0], [0, height_img], [width_img, height_img]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    img_output = cv2.warpPerspective(img, matrix, (width_img, height_img))
    img_cropped = cv2.resize(img_output, (width_img, height_img))
    # image_grayworld = ((img_cropped * (img_cropped.mean() / img_cropped.mean(axis=(0, 1)))).clip(0, 255).astype(int)).astype(np.uint8)
    image_grayworld = (img_cropped * (img_cropped.mean() / img_cropped.mean(axis=(0, 1)))).clip(0, 255).astype(np.uint8)
    return image_grayworld

# CONTOURS TRAIN
# def text_detection(img, contours_data):
#     tensor = np.empty((1,4))
#     for contour in contours_data[:,:]:
#         x, y, w, h = contour[0], contour[2], contour[1], contour[3]
#         cv2.rectangle(img, (x, y), (w, h), (255, 255, 0), 6)
#         tensor = np.append(tensor, [[x, y, w, h]], axis=0)
#     if len_contours.value > 1:
#         if len(l2) < 2010:   
#             l2.append((tensor[1:,:].astype(int), 'Karta Polaka (BACK)'))
#         else:
#             print(len(l2))

# CONTOURS CHECK
def text_detection(fr, img, contours_data):
    
    if len_contours.value == 0:
        st = 'detecting...'
        color = (0, 0, 255)
    else:
        color = (0, 255, 0)
        temp = np.empty((1,4))
        for contour in contours_data[:,:]:
            x, y, w, h = contour[0], contour[2], contour[1], contour[3]
            cv2.rectangle(img, (x, y), (w, h), (255, 255, 0), 6)
            if temp.shape[0] < 50: # 50 pr2[2]
                temp = np.append(temp, [[x, y, w, h]], axis=0)
                
        t = temp[1:,:].astype(int)
        t = np.pad(t, [(0, 50 - t.shape[0]),(0, 0)], mode='constant').T.reshape(1, -1) # 50 pr2[2]
        ind = clf.predict(t).argmax(1)[0]
    
    
        st = list(target_dict.keys())[list(target_dict.values()).index(ind)]
    cv2.putText(fr, st, (7, 70), font, 3, color, 7, cv2.LINE_AA)

def close():
    frame_memory.close()
    frame_memory.unlink()
    contours_memory.close()
    contours_memory.unlink()
    p1.terminate()
    cap.release()

def view(button,):
    frame_data = np.ndarray((height_img,width_img,3), dtype=np.uint8, buffer=frame_memory.buf)
    contours_data = np.ndarray((1,4), dtype=np.int32, buffer=contours_memory.buf)
    
    while True:
        _, frame = cap.read()

        frame_masked = preprocess(frame)
        frame_contours = contours(frame_masked, frame)

        # contours_size.value = frame_contours.size

        # try:
        #     print(contours_data[0], len_contours.value)
        # except:
        #     ()
        # time.sleep(3)

        
        if frame_contours.size != 0:
            img_warped = get_warp(frame, frame_contours)
            frame_data[:,:,:] = img_warped[:,:,:] # перенести под tmp.value==1
            contours_size.value = 1
            
            if tmp.value == 1:
                contours_data = np.ndarray((len_contours.value,4), dtype=np.int32, buffer=contours_memory.buf)
                tmp.value = 0

            frame_detect = text_detection(frame, img_warped, contours_data)
        
        else:
            img_warped = frame.copy()
            frame_data[:,:,:] = img_warped[0,0,0] # может убрать?
            len_contours.value = 0

        img_stack = cvzone.stackImages([
            frame,
            img_warped,
        ], 3, 1)

        _, frame = cv2.imencode('.jpeg', img_stack) 
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            close()
            display_handle.update(None)

display(stopButton,)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

None

ToggleButton(value=False, description='Stop')

In [67]:
p1.terminate()
frame_memory.close()
contours_memory.close()
frame_memory.unlink()
contours_memory.unlink()
cap.release()
time.sleep(0.1)
p1.is_alive()

False

In [48]:
print(thread.is_alive(), p1.is_alive())

False True


In [24]:
l2 = []

In [27]:
del l2[:10]

In [30]:
del l2[500:510]

In [33]:
del l2[1000:1010]

In [36]:
del l2[1500:1510]

In [37]:
len(l2)

2000

In [38]:
def data_prepare(itemlist, target_dict):
    s = 0
    len_dict = len(target_dict)
    
    for i in itemlist:
        if s < i[0].shape[0]:
            s = i[0].shape[0]
            
    ret_input = np.empty((1, s * 4))
    ret_target = np.empty((1, len_dict))
    eye = np.eye(len_dict)
            
    for i in itemlist:
        t = np.pad(i[0], [(0, s - i[0].shape[0]),(0, 0)], mode='constant')
        
        ret_input = np.vstack((ret_input, t.T.reshape(1, -1)))
        ret_target = np.vstack((ret_target , eye[target_dict[i[1]]]))
        
    return ret_input[1:,:], ret_target[1:,:], s

In [40]:
pr = data_prepare(l2, target_dict)

In [41]:
pr[2]

50

In [44]:
X_train, X_test, y_train, y_test = train_test_split(pr[0], pr[1], test_size=0.2)

In [45]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1600, 200), (400, 200), (1600, 4), (400, 4))

In [46]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=0)

In [47]:
accuracy_score(y_train, clf.predict(X_train)), accuracy_score(y_test, clf.predict(X_test))

(1.0, 1.0)

In [50]:
with open('model_0.pkl', 'wb') as file:  
    pickle.dump(clf, file)